In [16]:
import findspark
findspark.init('/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from matplotlib import pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
rdd = sc.textFile('./csv/Pre_100k.csv')
rdd = rdd.map(lambda x: x.split(','))
rdd.take(10)

[[u'3603', u'5.0', u'2838'],
 [u'3603', u'4.0', u'7527'],
 [u'523', u'1.0', u'6801'],
 [u'2055', u'5.0', u'254'],
 [u'1350', u'5.0', u'3979'],
 [u'1417', u'5.0', u'6297'],
 [u'1417', u'5.0', u'3264'],
 [u'2906', u'5.0', u'4006'],
 [u'2906', u'5.0', u'2403'],
 [u'2906', u'5.0', u'1888']]

In [3]:
dfrdd = rdd.map(lambda x: Row(userid = int(x[2]),
                              rating = float(x[1]),
                              itemid = int(x[0])))

In [4]:
df = spark.createDataFrame(dfrdd)
df.show()

+------+------+------+
|itemid|rating|userid|
+------+------+------+
|  3603|   5.0|  2838|
|  3603|   4.0|  7527|
|   523|   1.0|  6801|
|  2055|   5.0|   254|
|  1350|   5.0|  3979|
|  1417|   5.0|  6297|
|  1417|   5.0|  3264|
|  2906|   5.0|  4006|
|  2906|   5.0|  2403|
|  2906|   5.0|  1888|
|  2906|   5.0|  1485|
|  2906|   5.0|  4489|
|  2906|   4.0|  2984|
|  2906|   5.0|   137|
|  2906|   5.0|  2963|
|  2906|   5.0|   912|
|   589|   5.0|   644|
|   589|   5.0|   583|
|   589|   5.0|  3233|
|   589|   5.0|    14|
+------+------+------+
only showing top 20 rows



In [21]:
tdf = df.groupby('userid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.userid == df.userid).drop(tdf.userid)
df = df.withColumnRenamed('avg(rating)', 'mean')
df.show()

+----+------+------+------+
|mean|itemid|rating|userid|
+----+------+------+------+
| 5.0|   701|   5.0|    26|
| 4.0|  2332|   4.0|    29|
| 5.0|  1681|   5.0|   474|
| 5.0|  3432|   5.0|   964|
| 5.0|  2161|   5.0|  1677|
| 5.0|  1806|   5.0|  1697|
| 1.0|  2142|   1.0|  1806|
| 5.0|  1233|   5.0|  1950|
| 5.0|   258|   5.0|  2040|
| 5.0|  1118|   5.0|  2214|
| 5.0|   631|   5.0|  2250|
| 1.0|  1024|   1.0|  2453|
| 4.0|  1515|   4.0|  2509|
| 5.0|  2401|   5.0|  2529|
| 5.0|  3703|   5.0|  2927|
| 5.0|  3070|   5.0|  3091|
| 5.0|    68|   5.0|  3091|
| 5.0|  2930|   5.0|  3091|
| 4.0|   704|   4.0|  3506|
| 5.0|   595|   5.0|  3764|
+----+------+------+------+
only showing top 20 rows

